<a href="https://colab.research.google.com/github/mr-cri-spy/SLM/blob/main/Ask_Bot()_Vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install transformers faiss-cpu


In [21]:
import json
import numpy as np
import torch
import faiss
from transformers import AutoTokenizer, AutoModel


Load Embedding Model – IndicBERT

In [22]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Define Function to Get Embeddings

In [23]:
def get_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)
        return output.last_hidden_state.mean(dim=1).squeeze().numpy().astype("float32")


Prepare Sample FAQ Data

In [24]:
faq_pairs = [
    {"question": "What is Crisbee?", "answer": "Crisbee is an AI startup focusing on chatbot automation."},
    {"question": "What services do you offer?", "answer": "We offer AI chatbots, voice bots, and NLP solutions."},
    {"question": "Where are you located?", "answer": "We are based in Mysore city,karnataka state , country India."},
    {"question": "Do you support regional languages?", "answer": "Yes, we support Hindi, Kannada, and Tamil."}


]

# Split questions and answers
questions = [item["question"] for item in faq_pairs]
answers = [item["answer"] for item in faq_pairs]


Embed Questions and Build FAISS Index

In [25]:
question_embeddings = np.array([get_embedding(q) for q in questions]).astype("float32")

dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 Define ask_bot() Function

In [26]:
def ask_bot(query, k=1):
    query_vector = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_vector, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        answer = answers[idx]
        confidence = 1 / (1 + dist)  # lower distance → higher confidence
        results.append({
            "answer": answer,
            "confidence": round(confidence, 3)
        })

    return results[0] if k == 1 else results


Test the Bot (English + Kannada + Hindi)

In [27]:
# English
print(ask_bot("Where is your company located?"))

# Kannada
print(ask_bot("ನೀವು ಕನ್ನಡ ಬೆಂಬಲಿಸುತ್ತೀರಾ?"))

# Hindi
print(ask_bot("क्या आप हिंदी भाषा का समर्थन करते हैं?"))


{'answer': 'We are based in Mysore city,karnataka state , country India.', 'confidence': np.float32(0.091)}
{'answer': 'Crisbee is an AI startup focusing on chatbot automation.', 'confidence': np.float32(0.038)}
{'answer': 'Crisbee is an AI startup focusing on chatbot automation.', 'confidence': np.float32(0.035)}


Add Named Entity Recognition (NER)
NER can extract places (city, state, etc.)

In [30]:
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load("en_core_web_sm")

def extract_city(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["GPE"]:  # GPE = Geo-Political Entity (city/state/country)
            return ent.text
    return text  # fallback


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [31]:
raw_answer = ask_bot("Which city are you located in?")["answer"]
print("City:", extract_city(raw_answer))


City: Mysore
